# Propensity to Buy Credit Scoring Feed

This notebook can be scheduled to provide a continuous flow of data to your German Credit Risk model. Paste your Watson Machine Learning credentials below, and verify that the value of the **DEPLOYMENT_NAME** variable matches the name of your deployed German Credit Risk model. You can also set the maximum and minimum number of records that will be scored per run.

In [46]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

Requirement not upgraded as not directly required: docutils>=0.10 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client)


In [47]:
WML_CREDENTIALS = WML_CREDENTIALS ={
  "apikey": "wcPASQdXvtEPgA79lY4sYpCfiWAOoSNudy0hh24PMNbb",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/97deeb0b7e78431438a00a04f20580b7:e7e6948c-829b-4d59-ae09-3f991dfe52cf::",
  "iam_apikey_name": "auto-generated-apikey-f4fafb37-2a2d-477c-a9c6-0ff5400609f6",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/97deeb0b7e78431438a00a04f20580b7::serviceid:ServiceId-8e03beb2-f9d6-4351-b0e5-2add2a3437df",
  "instance_id": "e7e6948c-829b-4d59-ae09-3f991dfe52cf",
  "password": "d3538d26-7a6c-4e2f-b114-6a02041ce00b",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "f4fafb37-2a2d-477c-a9c6-0ff5400609f6"
}

In [48]:
DEPLOYMENT_NAME = "Propensity to Buy"
MIN_RECORDS = 1000
MAX_RECORDS = 1000

In [49]:
import json, random
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [50]:
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [51]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_biased.json

--2019-03-07 16:11:04--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_biased.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254940 (249K) [text/plain]
Saving to: ‘df_payload_biased.json’

100%[======================================>] 254,940     --.-K/s   in 0.01s   

2019-03-07 16:11:04 (18.3 MB/s) - ‘df_payload_biased.json’ saved [254940/254940]



In [52]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/deployments/90feae8c-ad4f-4f78-83d5-1d2e81b9c578/online


In [53]:
try:
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_biased.json
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')

file found


In [54]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

{'fields': ['INCOME', 'AGE_IN_YEARS', 'HAS_CHILDREN', 'RENTER', 'OWNER', 'GOOD_CREDIT', 'COLLEGE_DEGREE', 'AVG_DIST_FROM_HOME', 'MILLENIAL', 'MID_LEVEL_AUTO', 'DISTANCE_FROM_HOME', 'TRAVELER', 'TIME_OF_DAY', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[53417.0, 55.4458022178, 0.0, 0, 1, 1, 0, 13.6, 0, 1, 1, 0, 21, [53417.0, 55.4458022178, 0.0, 0.0, 1.0, 1.0, 0.0, 13.6, 0.0, 1.0, 1.0, 0.0, 21.0], [2.0006207323282483, 17.999379267671753], [0.10003103661641241, 0.8999689633835877], 1.0, 'NOT_BUY'], [80372.1807753994, 64.644957, 0.0, 0, 0, 0, 0, 2.8, 0, 0, 6, 0, 19, [13, [0, 1, 7, 10, 12], [80372.1807753994, 64.644957, 2.8, 6.0, 19.0]], [1.2777119600343327, 18.722288039965665], [0.06388559800171664, 0.9361144019982834], 1.0, 'NOT_BUY'], [87500.0002, 45.892055, 1.0, 0, 1, 1, 1, 32.04, 0, 0, 24, 0, 7, [87500.0002, 45.892055, 1.0, 0.0, 1.0, 1.0, 1.0, 32.04, 0.0, 0.0, 24.0, 0.0, 7.0], [17.851911364219063, 2.1480886357809394], [0.892595568210953, 0.10